# Creative Writing Workbench using LCEL (version 2)

Solution to **exercise #6**

Convert the Creative Writing Workbench code from *exercise #5* to use LCEL


Create a chain that has 4 steps.

1. Chain#1 : Generate 3 plan
2. Chain#2 : Vote the best plan out of the 3 plans
3. Chain#3 : Generate 3 writeups from the best plan
4. Chain#4 : Vote the best passage out of the 3 passages


**Hints**:

* To specify JSON formats in prompts use double '{{' instead of '{'  i.e., *{{   ...  }*
* You may need to adjust the prompt i.e., you cant use the prompts from earlier version as-is
* You may need to adjust the number of tokens returned from the model as a response
* If need be, use JSON output parser. Pydantic may also be used. Try out both and pick one that works best for you.
* Output indicators may be hard coded in the prompt as we will not be parsing the output from the LLM at each step

#### Google Colab
If you are running the code in Google colab, install the packages by uncommenting/running the cell below

* The API key file file will not be available
* You will be prompted to provide the Cohere API Token

Uncomment & run the code in the cell below:

In [ ]:
## The script is downloaded and run to setup the utils folder

# !curl -H "Accept: application/vnd.github.VERSION.raw" https://raw.githubusercontent.com/acloudfan/gen-ai-app-dev/main/Setup/gcsetup.sh  > gcsetup.sh
# !chmod u+x gcsetup.sh
# !./gcsetup.sh -l

## Setup the environment variable for API access

#### NOTE
You MUST change the path to your own environment file or provide the API key value

In [1]:
from dotenv import load_dotenv
import os
import sys
import warnings

from IPython.display import Markdown, JSON

warnings.filterwarnings("ignore")

# Load the file that contains the API keys
load_dotenv('C:\\Users\\raj\\.jupyter\\.env')

True

In [2]:
# Setting path so we can access the utils folder
sys.path.append('../')
sys.path.append('./')

from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableSequence
from operator import itemgetter
from utils.create_llm import create_google_llm, create_cohere_llm, create_gpt_llm
from langchain.globals import set_debug

from langchain.prompts import PromptTemplate
from langchain.chains  import LLMChain, SequentialChain

from IPython.display import Markdown, JSON

## Writeup instructions

In [3]:
writeup_instructions = """
Write a passage of under 100 words to describe our new product:
Javelin 20 High Performance Shoes.

The title must contain the sentence: A new you

The first sentence must contain a few of these words: motivation, workout, sports, attractive

The last few sentences must motivate the reader to visit out website http://acme.com/jav20

The passage must contain the phrase : trendsetter
"""

## LLM setup

In [4]:
# You may use different LLM  for each of the chains !!
# NOTE : the behavior will change as we are not adjusting the prompt
llm_chain_1 = create_google_llm(api_key_prompt=True)
llm_chain_2 = llm_chain_1
llm_chain_3 = llm_chain_1
llm_chain_4 = llm_chain_1



Key:  GOOGLE_API_KEY  already set in environment.


## 1. Create LLMChain #1   Generate 3 plans


In [5]:
ouput_passage_indicator = "Passage:"
ouput_plan_indicator = "Plan:"

template_1 = """
Generate three separate step by step plans for executing the given task with the following task instructions.

Task Instructions:
{task_instructions}

{ouput_plan_indicator}
Your plan here.

""" 

prompt_template_1 = PromptTemplate(
                        template = template_1,
                        input_variables = ["task_instructions"],
                        partial_variables = {"ouput_plan_indicator":ouput_plan_indicator}
)

# print(prompt_template_1.format(task_instructions=writeup_instructions))

In [6]:
# Setup a sequential chain with prompt & LLM
chain_1_sequence = prompt_template_1 |  llm_chain_1

# We will use the RunnableParallel so we can pass on the task_instructions as is to the output
generate_plan_chain_1 = RunnableParallel(
    task_instructions=itemgetter('task_instructions'), 
    proposed_plans = chain_1_sequence
)


#### Unit test chain#1
Verify the output from the chain - it should be a dictionary with 2 keys [task_instructions, proposed_plans]

In [7]:
# Response form the chain
response = generate_plan_chain_1.invoke({"task_instructions": writeup_instructions})

# Pretty print JSON
JSON(response) #['proposed_plans'])


<IPython.core.display.JSON object>

## 2. Vote the best plan out of the 3 plans

In [8]:
output_best_plan_indicator = "Selected Plan:"

template_2 = """
Three  expert marketing writers were asked to put together a plan for the following task. 
Your job is to pick the best plan.

Task Instructions:
{task_instructions}

Proposed Plans:
{proposed_plans}

Your out should be in following format.

{output_best_plan_indicator} 
Put the plan steps here.

"""

prompt_template_2 = PromptTemplate(
                        template = template_2,
                        input_variables = ["task_instructions", "proposed_plans"],
                        partial_variables = {"output_best_plan_indicator":output_best_plan_indicator}
)

# print(prompt_template_2.format(task_instructions=writeup_instructions, proposed_plans = "DUMMY PLANS"))

In [9]:
# Setup a sequential chain with prompt & LLM
chain_2_sequence = prompt_template_2 |  llm_chain_2

# We will use the RunnableParallel so we can pass on the task_instructions & proposed plans as is to the output
vote_on_plan_chain_2 = RunnableParallel(
    task_instructions=itemgetter('task_instructions'), 
    proposed_plans = itemgetter('proposed_plans'),
    best_voted_plan = chain_2_sequence
)

# vote_on_plan_chain_2 = prompt_template_2 | {"best_voted_plan": llm_chain_2}


#### Unit testing

Create a chain with chain#1 and chain#2, to check out how things are going.

OPTIONAL step - just to check how things are going

In [10]:
# Create a sequential chain with chain#1 and chain#2
chain_1_chain_2 =  generate_plan_chain_1  | vote_on_plan_chain_2


In [11]:
response = chain_1_chain_2.invoke({"task_instructions":writeup_instructions})

JSON(response) #['best_voted_plan'])

<IPython.core.display.JSON object>

## 3. Create 3 passages with best_voted_plan

In [12]:
template_3 = """
Create three passages using the following task instructions and the plan.

Task Instructions:
{task_instructions}

Plan:
{best_voted_plan}

Your output will be in following format.

{ouput_passage_indicator}
Put the passage here.

"""

prompt_template_3 = PromptTemplate(
    template = template_3,
    input_variables = ["best_voted_plan"],
    partial_variables = {"ouput_passage_indicator" : ouput_passage_indicator, "task_instructions": writeup_instructions},
)

# print(prompt_template_3.format(best_voted_plan="DUMMY PLAN"))

In [13]:
# Setup a sequential chain with prompt & LLM
chain_3_sequence = prompt_template_3 |  llm_chain_3

# We will use the RunnableParallel so we can pass on the task_instructions & proposed plans as is to the output
create_3_passages_chain_3 = RunnableParallel(
    task_instructions=itemgetter('task_instructions'), 
    proposed_plans = itemgetter('proposed_plans'),
    best_voted_plan = itemgetter('best_voted_plan'),
    passages = chain_3_sequence
)

#### Unit testing
Check with a sequence of chain#1, chain#2, chain#3


In [14]:
# Create a sequential chain with chain#1 and chain#2
chain_1_chain_2_chain_3 =  generate_plan_chain_1  | vote_on_plan_chain_2 | create_3_passages_chain_3

In [15]:
response = chain_1_chain_2_chain_3.invoke({"task_instructions":writeup_instructions})

JSON(response) 

<IPython.core.display.JSON object>

## 4. Vote the best passage

In [16]:

output_best_passage_indicator = "Selected Passage:"

template_4 = """
Three experts were given the following task.

Task:
{task_instructions}

You need to review the 3 passages and identify ONE best passage out of the three. 

Proposed Passages:
{passages}

Your output will be in following format.

{output_best_passage_indicator}
Put the passage here.

"""

prompt_template_4 = PromptTemplate(
    template = template_4,
    input_variables = ["passages"],
    partial_variables = {"task_instructions": writeup_instructions, "output_best_passage_indicator" : output_best_passage_indicator, },
)

# print(prompt_template_4.format(passages = "DUMMY PASSAGES"))

In [17]:
# Setup a sequential chain with prompt & LLM
chain_4_sequence = prompt_template_4 |  llm_chain_4

# We will use the RunnableParallel so we can pass on the task_instructions & proposed plans as is to the output
create_creative_passage_chain_4 = RunnableParallel(
    task_instructions=itemgetter('task_instructions'), 
    proposed_plans = itemgetter('proposed_plans'),
    best_voted_plan = itemgetter('best_voted_plan'),
    passages = itemgetter('passages'),
    creative_passage = chain_4_sequence
)

In [18]:
## 5. Create a sequence from all chains

creative_writing_chain = generate_plan_chain_1 | vote_on_plan_chain_2 | create_3_passages_chain_3 | create_creative_passage_chain_4

## 5. End-to-end testing
Check with a sequence of chain#1, chain#2, chain#3

In [19]:
response = creative_writing_chain.invoke({"task_instructions":writeup_instructions})
JSON(response)

<IPython.core.display.JSON object>

## 6. Create the graph

In [ ]:
# !pip install grandalf

In [21]:
## Checkout the final layout

# creative_writing_chain.get_graph().print_ascii()

                                          +-------------------------------------------------+                                    
                                          | Parallel<task_instructions,proposed_plans>Input |                                    
                                          +-------------------------------------------------+                                    
                                                           ***           ***                                                     
                                                         **                 **                                                   
                                                       **                     **                                                 
                                            +----------------+                  **                                               
                                            | PromptTemplate |                   *        